# Coding Homework 3: Federated Learning

In this notebook you'll implement a minimal **federated averaging (FedAvg)** algorithm on MNIST using PyTorch.

There are four main exercises:

1. Local client training (`fedavg_train_local`)
2. Model aggregation (`average_state_dicts`)
3. The federated training loop (`federated_train`)
4. The centralized training loop (`naive_train`)


Run the cells in order and complete all **TODO** blocks (the "YOUR CODE/ANSWER HERE" lines).

In [ ]:
import torch
import random
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from torchvision import datasets, transforms
from collections import OrderedDict
import numpy as np
import copy
import matplotlib.pyplot as plt

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Federated learning hyperparameters
num_clients = 10
batch_size = 64
local_epochs = 1
num_rounds = 100
frac_clients = 0.7  # fraction of clients that participate in each round
lr = 0.1

## Load and partition the dataset

We'll use the classic MNIST digit classification dataset and split the training set
into several disjoint subsets, one per client.

In [ ]:
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Split the training dataset into num_clients subsets
data_per_client = len(train_dataset) // num_clients
lengths = [data_per_client] * num_clients
# Add any remainder to the last client
lengths[-1] += len(train_dataset) - sum(lengths)

client_datasets = random_split(train_dataset, lengths)
client_loaders = [
    DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in client_datasets
]

print(f"Total train samples: {len(train_dataset)}")
for i, ds in enumerate(client_datasets):
    print(f"Client {i}: {len(ds)} samples")

## Define a simple model

We'll use a small fully-connected neural network that flattens the 28x28 images
and predicts one of 10 digit classes.

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 10),
        )

    def forward(self, x):
        return self.net(x)


global_model = SimpleNet().to(device)

## Evaluation helper

We'll use this function to evaluate the **global** model on the centralized test set.

In [ ]:
def evaluate(model, data_loader, device=device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            total_loss += loss.item() * x.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


loss, acc = evaluate(global_model, test_loader, device)
print(f"Untrained model -> loss: {loss:.4f}, acc: {acc:.4f}")

## Exercise 1: Local client training

Implement `fedavg_train_local`, which takes the current **global model** and trains a copy
of it on a single client's local data for a few epochs.

Steps to implement:

1. Make a copy of the input model so we don't modify it in-place.
2. Move it to the correct device.
3. Create an optimizer and a cross-entropy loss function.
4. Run standard supervised training for the specified number of local epochs.
5. Return the trained model's `state_dict`.

In [ ]:
def fedavg_train_local(model, train_loader, epochs=local_epochs, lr=lr, device=device):
    """
    Train a copy of the given global model on a single client's data.

    Args:
        model: the global model (nn.Module) to start from.
        train_loader: DataLoader for this client's local dataset.
        epochs: number of local epochs.
        lr: learning rate.
        device: "cpu" or "cuda".

    Returns:
        state_dict (dict): the trained model parameters for this client.
    """
    # Make a copy of the model so we don't modify the original in-place
    local_model = copy.deepcopy(model).to(device)

    optimizer = optim.SGD(local_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    ### YOUR CODE HERE ###
    
    # Return a (detached) copy of the trained parameters
    return copy.deepcopy(local_model.state_dict())

In [ ]:
local_state_dict = fedavg_train_local(global_model, client_loaders[0], epochs=1, lr=0.1, device=device)
local_model = copy.deepcopy(global_model)
local_model.load_state_dict(local_state_dict)
# Evaluate the test model on the test dataset
test_loss, test_acc = evaluate(local_model, test_loader, device)
print(f"One local round -> loss: {test_loss:.4f}, acc: {test_acc:.4f}")


## Exercise 2: Parameter aggregation (FedAvg)

Implement `average_state_dicts`, which computes the **element-wise average**
of a list of model parameter dictionaries. This is the core of the
Federated Averaging (FedAvg) algorithm.

Steps to implement:

1. Create an empty `OrderedDict` to hold the averaged parameters.
2. For each parameter key, compute the mean of that parameter
   across all client `state_dict`s.
3. Store the averaged tensor in the new `OrderedDict` and return it.

In [ ]:
def average_state_dicts(state_dicts):
    """
    Compute the element-wise average of a list of model state_dicts.

    Args:
        state_dicts: list of state_dict objects from different clients.

    Returns:
        A new state_dict containing the average parameters.
    """
    avg_state_dict = OrderedDict()

    ### YOUR CODE HERE ###

    return avg_state_dict

## Exercise 3: Federated training loop

Implement the main `federated_train` function, which performs several
communication rounds:

1. Sample a subset of clients.
2. Send the current global model to each selected client.
3. Each client trains locally and returns updated model weights.
4. Aggregate client weights using `average_state_dicts`.
5. Update the global model and evaluate it on the test set.

In [ ]:
def federated_train(
    global_model,
    client_loaders,
    num_rounds=num_rounds,
    frac_clients=frac_clients,
    local_epochs=local_epochs,
    lr=lr,
    device=device,
    train_local_fn=fedavg_train_local,
    **kwargs
):
    """
    Run the federated averaging algorithm.

    Args:
        global_model: the model that will be shared and updated.
        client_loaders: list of DataLoaders, one per client.
        num_rounds: number of communication rounds.
        frac_clients: fraction of clients to sample each round (0 < frac_clients <= 1).
        local_epochs: number of local training epochs.
        lr: learning rate for local training.
        device: computation device.

    Returns:
        A history list with (round, test_loss, test_acc).
    """
    history = []
    num_clients = len(client_loaders)
    global_model.to(device)

    for rnd in range(1, num_rounds + 1):
        ### YOUR CODE HERE ###

        test_loss = None
        test_acc = None

        # Evaluate on the test set
        test_loss, test_acc = evaluate(global_model, test_loader, device)
        history.append((rnd, test_loss, test_acc))
        print(f"Round {rnd:02d}: test loss={test_loss:.4f}, acc={test_acc:.4f}")

    return history

## Run federated training

After completing all TODOs above, run this cell to train the federated model
and inspect the results.

In [ ]:
history = federated_train(global_model, client_loaders, num_rounds=num_rounds, local_epochs=local_epochs, train_local_fn=fedavg_train_local)
print("Training finished!")

Now, let's compare with centralized training without FedAvg

## Exercise 4: Centralized training loop

Implement `naive_train`, which takes the **centralized model** and trains it on the whole dataset. Run standard supervised training for the specified number of epochs. Hint: this should be similar to exercise 1.

In [ ]:
def naive_train(model, full_dataloader, epochs, lr=lr, device=device):
    """
    Train a model on the whole dataset.

    Args:
        model: the centralized model (nn.Module) to train.
        full_dataloader: DataLoader for the full dataset.
        epochs: number of local epochs.
        lr: learning rate.
        device: "cpu" or "cuda".

    Returns:
        A history list with (round, test_loss, test_acc).
    """
    
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    naive_training_history = []

    model.train()
    for epoch in range(epochs):
        ### YOUR CODE HERE ###

        loss, acc = evaluate(model, test_loader, device)
        naive_training_history.append((epoch + 1, loss, acc))
        print(f"Naive training epoch {epoch + 1:02d}: test loss={loss:.4f}, acc={acc:.4f}")

    # Evaluate the test model on the test dataset
    test_loss, test_acc = evaluate(model, test_loader, device)
    print(f"Naive model training without FedAvg -> loss: {test_loss:.4f}, acc: {test_acc:.4f}")

    return naive_training_history

In [ ]:
full_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
centralized_model = SimpleNet().to(device)
num_naive_training_epochs = num_rounds * local_epochs
naive_training_history = naive_train(centralized_model, full_dataloader, num_naive_training_epochs)
print("Training finished!")

In [ ]:
# Plot the history of federated training vs full training
def plot_history(history_dict):
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    for name, history in history_dict.items():
        rounds, losses, accs = zip(*history)
        plt.plot(rounds, accs, label=name)
    plt.xlabel("Rounds")
    plt.ylabel("Test Accuracy")
    plt.title("Test Accuracy Comparison")
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    for name, history in history_dict.items():
        rounds, losses, accs = zip(*history)
        plt.plot(rounds, losses, label=name)
    plt.xlabel("Rounds")
    plt.ylabel("Test Loss")
    plt.title("Test Loss Comparison")
    plt.legend()
    
    plt.show()

all_history_dict = {
    "FedAvg": history,
    "Full Training": naive_training_history
}
plot_history(all_history_dict)


What do you observe when comparing the training loss and accuracy between local training and FedAvg? Please provide an explaination for why this is happening.

--- YOUR ANSWER HERE ---

# Non-IID Scenario

In the previous setting, we saw how FedAvg compares to naive local training when the datasets held by clients are relatively homogeneous. Now let’s make the problem more challenging by considering a heterogeneous scenario.

Imagine we have 10 hospitals, each specializing in a different medical domain and therefore collecting more data in that specific area. These hospitals would like to collaborate to build a single model that performs well across all domains. Federated learning is a natural way to enable this kind of collaboration without sharing raw data.

In our toy example, we’ll treat each MNIST class as a different medical domain. We’ll construct a synthetic, non-iid dataset that mimics the kind of label skew you might see when each client primarily observes data from its own domain.


In [ ]:
num_classes = 10
labels = np.array(train_dataset.targets)

class_indices = [np.where(labels == k)[0] for k in range(num_classes)]
for idx in class_indices:
    np.random.shuffle(idx)

def dirichlet_partition(class_indices, num_clients, alpha=0.1):
    """
    Non-IID label skew using Dirichlet distribution.
    Smaller alpha -> stronger skew (harder for FedAvg).
    """
    client_indices = [[] for _ in range(num_clients)]

    for k in range(len(class_indices)):
        idx_k = class_indices[k]
        np.random.shuffle(idx_k)

        # Draw proportions for this class over clients
        proportions = np.random.dirichlet(alpha * np.ones(num_clients))

        # Compute split points
        split_points = (np.cumsum(proportions) * len(idx_k)).astype(int)[:-1]
        splits = np.split(idx_k, split_points)

        for cid, split in enumerate(splits):
            client_indices[cid].extend(split.tolist())

    # Shuffle within each client
    for cid in range(num_clients):
        np.random.shuffle(client_indices[cid])

    return client_indices

# Strong heterogeneity: alpha = 0.1 (try 0.05 or 0.01 for even harder)
alpha = 0.1
client_indices = dirichlet_partition(class_indices, num_clients, alpha=alpha)

# Add client-specific label noise
class NoisyClientDataset(Dataset):
    """Wraps MNIST + indices + precomputed noisy labels."""
    def __init__(self, base_dataset, indices, noisy_labels):
        self.base_dataset = base_dataset
        self.indices = np.array(indices)
        self.noisy_labels = np.array(noisy_labels)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        x, _ = self.base_dataset[self.indices[idx]]  # ignore original label
        y = int(self.noisy_labels[idx])
        return x, y

def apply_noise_for_client(labels, indices, noise_type, num_classes, noise_level=0.4, rng=None):
    """
    Create noisy labels for one client.
    noise_type:
        - "none"         : no label noise
        - "symmetric"    : flip to a random label with prob p
        - "pairflip"     : flip y -> (y+1) mod K with prob p
        - "single_class" : only corrupt one chosen source class to a target class
    """
    if rng is None:
        rng = np.random.default_rng()

    y = labels[indices].copy()

    if noise_type == "none":
        return y

    if noise_type == "symmetric":
        mask = rng.random(len(y)) < noise_level
        random_labels = rng.integers(0, num_classes, size=mask.sum())
        y[mask] = random_labels

    elif noise_type == "pairflip":
        mask = rng.random(len(y)) < noise_level
        y[mask] = (y[mask] + 1) % num_classes

    elif noise_type == "single_class":
        # Pick a source class and a target class (different) for this client
        source_class = int(rng.integers(0, num_classes))
        target_class = int((source_class + rng.integers(1, num_classes)) % num_classes)
        mask = (y == source_class) & (rng.random(len(y)) < noise_level)
        y[mask] = target_class

    else:
        raise ValueError(f"Unknown noise_type: {noise_type}")

    return y

# Define a set of noise types and cycle through them across clients
noise_types = ["none", "symmetric", "pairflip", "single_class"]

noniid_client_datasets = []
rng = np.random.default_rng(42)

for cid, idxs in enumerate(client_indices):
    noise_type = noise_types[cid % len(noise_types)]
    noisy_labels = apply_noise_for_client(
        labels,
        np.array(idxs),
        noise_type=noise_type,
        num_classes=num_classes,
        noise_level=0.4,   # higher -> harder
        rng=rng
    )
    ds = NoisyClientDataset(train_dataset, idxs, noisy_labels)
    noniid_client_datasets.append(ds)

noniid_client_loaders = [
    DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in noniid_client_datasets
]

print(f"Total train samples: {len(train_dataset)}")
for i, ds in enumerate(noniid_client_datasets):
    # extract noisy label distribution
    noisy_y = ds.noisy_labels
    unique, counts = np.unique(noisy_y, return_counts=True)
    dist_str = ", ".join(f"{u}:{c}" for u, c in zip(unique, counts))
    print(f"Client {i}: {len(ds)} samples | noise_type={noise_types[i % len(noise_types)]} | label distribution -> {dist_str}")


Let's try FedAvg with the non-iid datasets.

In [ ]:
global_model_noniid = SimpleNet().to(device)
non_iid_history = federated_train(global_model_noniid, noniid_client_loaders, num_rounds=num_rounds, local_epochs=local_epochs, train_local_fn=fedavg_train_local)
all_history_dict[f"FedAvg Non-IID"] = non_iid_history

In [ ]:
plot_history(all_history_dict)

What do you observe when comparing the training loss and accuracy between FedAvg in the IID scenario and the non IID scenario? What may be contributing to what you see?

----- YOUR ANSWER HERE -----